In [1]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-vhg6w4b4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-vhg6w4b4
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install gdown


In [3]:
!pip install tqdm whisper
!pip install jiwer tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.3 MB/s eta 0:00:00


In [4]:
import whisper
import os
import pandas as pd
from tqdm.notebook import tqdm
#from jiwer import wer

In [5]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import os

# Define Google Drive folder paths
kb_test_hi_path = "/content/drive/MyDrive/archive-7/kb_test_hi"
kb_valid_hi_path = "/content/drive/MyDrive/archive-7/kb_valid_hi"
transcription_test_file=("/content/drive/MyDrive/archive-7/kb_test_pairs.txt")
transcription_valid_file=("/content/drive/MyDrive/archive-7/kb_valid_pairs.txt")

# Check if the folders exist
if not os.path.exists(kb_test_hi_path):
    raise FileNotFoundError(f"Path '{kb_test_hi_path}' does not exist in Google Drive.")

if not os.path.exists(kb_valid_hi_path):
    raise FileNotFoundError(f"Path '{kb_valid_hi_path}' does not exist in Google Drive.")


In [7]:
model = whisper.load_model("tiny")

100%|█████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 81.8MiB/s]


In [8]:


# Define output files for Whisper transcriptions
whisper_transcription_test_file = 'whisper_transcription_test.txt'
whisper_transcription_valid_file = 'whisper_transcription_valid.txt'

# Transcription options
transcription_options = {"language": "hi", "task": "transcribe"}

# Transcribe the audio files from the test folder
for audio_file in tqdm(os.listdir(kb_test_hi_path)):
    if audio_file.endswith(".wav"):
        audio_path = os.path.join(kb_test_hi_path, audio_file)
        transcription = model.transcribe(audio_path, **transcription_options)["text"]
        with open(whisper_transcription_test_file, 'a') as f:
            f.write(transcription + '\n')

# Transcribe the audio files from the valid folder
#for audio_file in tqdm(os.listdir(kb_valid_hi_path)):
 #   if audio_file.endswith(".wav"):
  #      audio_path = os.path.join(kb_valid_hi_path, audio_file)
   #     transcription = model.transcribe(audio_path, **transcription_options)["text"]
    #    with open(whisper_transcription_valid_file, 'a') as f:
     #       f.write(transcription + '\n')


  0%|          | 0/3133 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not sup

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
with open(transcription_test_file, 'r') as ref:
    test_reference = ref.readlines()

#with open(transcription_valid_file, 'r') as ref:
 #  test_reference = ref.readlines()

# Load Whisper transcriptions
with open(whisper_transcription_test_file, 'r') as hyp:
    test_transcriptions = hyp.readlines()

#with open(whisper_transcription_valid_file, 'r') as hyp:
 #   valid_transcriptions = hyp.readlines()

In [28]:
# Function to remove blank lines and extra spaces
def clean_lines(lines):
    return [line.strip() for line in lines if line.strip()]  # Strip leading/trailing spaces and remove blank lines

# Clean reference and hypothesis files
with open(transcription_test_file, 'r') as f:
    test_reference = clean_lines(f.readlines())

with open(whisper_transcription_test_file, 'r') as f:
    test_transcriptions = clean_lines(f.readlines())


In [31]:
# Define a function to truncate lines to the shorter length
def truncate_to_match(reference, hypothesis):
    """
    Truncate the reference to match the length of the hypothesis.
    """
    min_length = min(len(reference), len(hypothesis))
    return reference[:min_length], hypothesis[:min_length]

# Clean and truncate the reference and hypothesis
with open(transcription_test_file, 'r') as f:
    test_reference = clean_lines(f.readlines())  # Apply cleaning as described earlier

with open(whisper_transcription_test_file, 'r') as f:
    test_transcriptions = clean_lines(f.readlines())  # Apply cleaning as described earlier

# Truncate reference to match hypothesis length
test_reference, test_transcriptions = truncate_to_match(test_reference, test_transcriptions)

# Ensure that after truncation, the counts match
if len(test_reference) != len(test_transcriptions):
    raise ValueError(f"After truncation, counts still do not match: Reference has {len(test_reference)}, Hypothesis has {len(test_transcriptions)}.")


In [13]:
from jiwer import wer

In [32]:
test_wer = wer(test_reference, test_transcriptions)
#valid_wer = wer(train_reference, valid_transcriptions)

print(f"Word Error Rate (WER) - Test Set: {test_wer:.4f}")
#print(f"Word Error Rate (WER) - Valid Set: {valid_wer:.4f}")

Word Error Rate (WER) - Test Set: 3.4416
